In [1]:
# from thetaray.api.tools.metadata import trigger_metadata_sync
# trigger_metadata_sync(allow_type_changes=True)

In [2]:
import datetime
from dateutil.relativedelta import relativedelta
import json
import logging
from pyspark.sql import DataFrame, Window, functions as f
from pyspark.sql import SQLContext
from pyspark.sql.types import LongType
import yaml

from common.libs import dates as dates_lib
from common.libs import features_discovery
from common.libs.features_executor import FeaturesExecutor
from common.libs.feature_engineering import max_look_back_monthly_features, max_look_back_daily_weekly_features
from common.libs.zscore import enrich_with_z_score
from common.factory.wrangling_execution_strategy import get_wrangling_execution_strategy
from common.factory.eval_flow_definition import get_evaluation_flow_definition
from common.factory.domain_definition import get_domain_definition
from common.notebook_utils.wrangling.wrangling_execution_strategy import WranglingExecutionStrategy
from common.definitions.domain import DomainDefinition
from common.definitions.eval_flow import EvaluationFlowDefinition
from common.libs.context_utils import get_dataset

from thetaray.api.context import init_context
from thetaray.api.dataset import dataset_functions
from thetaray.api.solution import IngestionMode
from thetaray.common import Constants
from thetaray.common.data_environment import DataEnvironment

logging.getLogger().handlers[0].setFormatter(logging.Formatter(fmt='%(levelname)s: %(asctime)s @ %(message)s',datefmt='%Y-%m-%d %H:%M:%S'))
logging.basicConfig(level=logging.INFO)

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


from thetaray.api.context import init_context
import datetime
from thetaray.common import Constants

from common.libs.config.loader import load_config
from common.libs.config.basic_execution_config_loader import BasicExecutionConfig, DevBasicExecutionConfig
from common.libs.context_utils import is_run_triggered_from_airflow



with open('/thetaray/git/solutions/domains/demo_fuib/config/spark_config.yaml') as spark_config_file:
    spark_config = yaml.load(spark_config_file, yaml.FullLoader)['spark_config_a']

execution_date=datetime.datetime(1970, 1, 1)

context = init_context(domain='demo_fuib',
                       execution_date=execution_date,
                       spark_conf=spark_config,
                       spark_master='local[*]',
                       allow_type_changes=True)

spark = context.get_spark_session()
sc = SQLContext(spark)
params = context.parameters
print(f"Spark UI URL: {context.get_spark_ui_url()}")

print(json.dumps(params, indent=4))

/thetaray/venv/lib64/python3.11/site-packages/starlette/config.py:60: UserWarning: Config file '.env' not found.
  warnings.warn(f"Config file '{env_file}' not found.")
/thetaray/venv/lib64/python3.11/site-packages/starlette/config.py:60: UserWarning: Config file '.env' not found.
  warnings.warn(f"Config file '{env_file}' not found.")
2025-09-15 10:09:51,900:INFO:thetaray.common.logging:start loading solution.....[ load_risks=True , solution_path=/thetaray/git/solutions/domains , settings_path=/thetaray/git/solutions/settings ]
2025-09-15 10:09:51,909:WARNING:thetaray.common.logging:Encryption is enabled on dataset demo_ret_smb_customer_insights, but data encryption is disabled in deployment
2025-09-15 10:09:51,924:WARNING:thetaray.common.logging:Encryption is enabled on dataset demo_ret_indiv_customer_insights, but data encryption is disabled in deployment
2025-09-15 10:09:52,068:WARNING:thetaray.common.logging:Encryption is enabled on dataset demo_digital_wallets_customer_insights, 

Added `alias` successfully.


INFO: 2025-09-15 10:10:29 @ === Finished updating schema for Evaluation Flows on Minio ===


Added `alias` successfully.
Spark UI URL: https://jupyterhub-platform-thetalab.sonar.thetaray.cloud/user/andre.gutnik/proxy/4041/jobs/
{}


/opt/spark/python/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
from thetaray.api.evaluation import evaluate
from thetaray.api.evaluation import write_evaluated_activities, publish_evaluated_activities
from thetaray.api.dataset import dataset_functions
from common.factory.eval_flow_definition import get_evaluation_flow_definition, EvaluationFlowDefinition

# eval_flow_def: EvaluationFlowDefinition = get_evaluation_flow_definition(basic_execution_config)

In [4]:
from thetaray.common.data_environment import DataEnvironment
import pyspark.sql.functions as f
ds_train = dataset_functions.read(context, 'customer_monthly', data_environment=DataEnvironment.PUBLIC).drop('tr_timestamp') 
ds_train.count()
 #TODO: complete

18000

In [5]:
# ds_train = ds_train.filter(f.col('year_month')=='2025-06-01')
# ds_train.count()

In [6]:
# ds_train = ds_train.filter(f.col('year_month')>'2024-07-01')
# ds_train.count()

In [7]:
evaluated_df = evaluate(context,'cust_month_ef', ds_train, data_environment=DataEnvironment.PUBLIC)
evaluated_df.count()

INFO: 2025-09-15 10:10:33 @ Applying backwards compatibility patches
2025-09-15 10:10:34.179211: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-15 10:10:34.181036: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-15 10:10:34.184065: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-15 10:10:34.192397: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757931034.205771   20960 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin

INFO: 2025-09-15 10:10:36 @ Rolling back backwards compatibility patches
INFO: 2025-09-15 10:10:36 @ Rolled back backwards compatibility patches
INFO: 2025-09-15 10:10:36 @ Applying backwards compatibility patches
INFO: 2025-09-15 10:10:36 @ Applied backwards compatibility patches


INFO: 2025-09-15 10:10:36 @ Rolling back backwards compatibility patches
INFO: 2025-09-15 10:10:36 @ Rolled back backwards compatibility patches
/thetaray/venv/lib64/python3.11/site-packages/starlette/config.py:60: UserWarning: Config file '.env' not found.
  warnings.warn(f"Config file '{env_file}' not found.")
/thetaray/venv/lib64/python3.11/site-packages/starlette/config.py:60: UserWarning: Config file '.env' not found.
  warnings.warn(f"Config file '{env_file}' not found.")
/thetaray/venv/lib64/python3.11/site-packages/starlette/config.py:60: UserWarning: Config file '.env' not found.
  warnings.warn(f"Config file '{env_file}' not found.")
/thetaray/venv/lib64/python3.11/site-packages/starlette/config.py:60: UserWarning: Config file '.env' not found.
  warnings.warn(f"Config file '{env_file}' not found.")
2025-09-15 10:10:41,881:INFO:thetaray.common.logging:Applying backwards compatibility patches
2025-09-15 10:10:41,886:INFO:thetaray.common.logging:Applying backwards compatibility

18000

In [8]:
# evaluated_df.filter(f.col('algo_score')>0.5).groupBy('year_month').count().orderBy('year_month',ascending=False).show()

In [9]:
# evaluated_df.filter(f.col('customer_id')=='C_G7HWMEPAHJ').filter(f.col('year_month')=='2025-06-01').select('algo_pattern','algo_score').show(truncate=False)

In [10]:
# evaluated_df.filter(f.col('algo_score')>0.5).filter(f.col('year_month')=='2025-06-01').select('algo_pattern','algo_score').show(truncate=False)

In [11]:
from thetaray.api.evaluation import unblock_evaluation_for_execution_date
unblock_evaluation_for_execution_date(context, 'cust_month_ef', data_environment=DataEnvironment.PUBLIC)

True

In [12]:
write_evaluated_activities(context, evaluated_df, 'cust_month_ef', data_environment=DataEnvironment.PUBLIC)

25/09/15 10:10:48 ERROR FileUtils: Failed to delete s3a://thetaray-sonar/warehouse/public.db/cust_month_ef/tr_year=1970/tr_month=1/tr_day=1/tr_date=1970_01_01_00_00_00/tr_partition=2024_07
25/09/15 10:10:48 ERROR FileUtils: Failed to delete s3a://thetaray-sonar/warehouse/public.db/cust_month_ef/tr_year=1970/tr_month=1/tr_day=1/tr_date=1970_01_01_00_00_00/tr_partition=2024_08
25/09/15 10:10:48 ERROR FileUtils: Failed to delete s3a://thetaray-sonar/warehouse/public.db/cust_month_ef/tr_year=1970/tr_month=1/tr_day=1/tr_date=1970_01_01_00_00_00/tr_partition=2024_09
25/09/15 10:10:48 ERROR FileUtils: Failed to delete s3a://thetaray-sonar/warehouse/public.db/cust_month_ef/tr_year=1970/tr_month=1/tr_day=1/tr_date=1970_01_01_00_00_00/tr_partition=2024_10
25/09/15 10:10:48 ERROR FileUtils: Failed to delete s3a://thetaray-sonar/warehouse/public.db/cust_month_ef/tr_year=1970/tr_month=1/tr_day=1/tr_date=1970_01_01_00_00_00/tr_partition=2024_11
25/09/15 10:10:49 ERROR FileUtils: Failed to delete s3a

In [13]:
publish_evaluated_activities(context, 'cust_month_ef', data_environment=DataEnvironment.PUBLIC)

INFO: 2025-09-15 10:11:06 @ _publish_evaluated_activities
           [evaluation_flow_identifier=cust_month_ef,
            date range [context 1970-01-01 00:00:00, from_job_ts None, to_job_ts None]
            data_environment=DataEnvironment.PUBLIC 
INFO: 2025-09-15 10:11:10 @ finished publishing records for evaluation flow cust_month_ef
INFO: 2025-09-15 10:11:10 @ models_metadata_df.rdd.isEmpty(): False
INFO: 2025-09-15 10:11:10 @ models_metadata_df.count(): 5


True

In [15]:
context.close()